In [2]:
# Import libraries
import pandas as pd
import requests
import json
import time
import re
from datetime import datetime, timedelta
from scipy.stats import linregress
import matplotlib.pyplot as plt
from pprint import pprint
import sys
# This must be changed if exported to .py file
sys.path.append("../common")
from api_keys_local_SW import nyt_API_key

In [3]:
# import JSON as object
with open('test/articlesearch_gamestop_202102.json', encoding="UTF-8") as j:
  json_data = json.load(j)

In [4]:
# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
pprint(json_data["response"])

                          'width': 4500},
                          {'caption': None,
                           'credit': None,
                           'crop_name': 'threeByTwoMediumAt2X',
                           'height': 1000,
                           'legacy': {},
                           'rank': 0,
                           'subType': 'threeByTwoMediumAt2X',
                           'subtype': 'threeByTwoMediumAt2X',
                           'type': 'image',
                           'url': 'images/2021/02/04/business/04economy-briefing-markets/merlin_171412242_4f224a62-0b32-403a-b2b0-a6738136fae6-threeByTwoMediumAt2X.jpg',
                           'width': 1500},
                          {'caption': None,
                           'credit': None,
                           'crop_name': 'threeByTwoSmallAt2X',
                           'height': 400,
                           'legacy': {},
                           'rank': 0,
                           'subTy

In [5]:
main_headline = json_data["response"]["docs"][0]["headline"]["main"]
print(main_headline)

The Hopes That Rose and Fell With GameStop


In [6]:
# RULES
# MAX 5 terms in search terms list
# MAX 30 pages per term
# SLEEP AT 6

# set up utilities/params
api_key = nyt_API_key
search_term = "gamestop"
#search_term_l = ["gamestop", "term2", "term3", "term4", "term5"]
begin_date = "20201101"
end_date = "20210220"
current_page_counter = 0
page_count_limit = 30

# set up lists to append to
abstract_l = []
web_url_l = []
main_headline_l = []
keywords_l = []
pub_date_l =  []
document_type_l = []
type_of_material_l = []
nyt_id_l = []
word_count_l = []


# for each search term??
# for each query result page
#CHANGE RANGE TO ~30 FOR LIVE RUN
for x in range (0,2):
    # sleep to follow API rules
    time.sleep(6)
    url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={search_term}&begin_date={begin_date}&end_date={end_date}&page={current_page_counter}&api-key={api_key}'
    try:
        # make query
        requestObject = requests.get(url)
        response = json.loads(requestObject.text)
        print()
        print("The status code is {}".format(requestObject.status_code) + " for page " + str(current_page_counter) + " of: gamestop")
        print(requestObject.url[0:-41])
    except:
        print()
        print("Invalid results. Moving to next request.")             
        pageHitCount = 0
        break
    if requestObject.status_code == 200:
        #for each article's docs section
        for x in response["response"]["docs"]:
            abstract_l.append(x["abstract"])
            web_url_l.append(x["web_url"])
            main_headline_l.append(x["headline"]["main"])
            pub_date_l.append(x["pub_date"])
            document_type_l.append(x["document_type"])
            type_of_material_l.append(x["type_of_material"])
            nyt_id_l.append(x["_id"])
            word_count_l.append(x["word_count"])
            #find keywords
            tuple_list = []
            for ky in x["keywords"]:
                tuple_list.append(ky["value"])
            keywords_l.append(tuple(tuple_list))
        #loop for this article is finished
        #increase offset
        current_page_counter += 1
        #loop for this page is finished
    #check for done condition
    if current_page_counter > page_count_limit:
        #if done, break out of loop
        break

# keywords_l = []
# for x in artlist:
#     tuple_list = []
#     count = x["keywords"]
#     for y in count:
#         tuple_list.append(y["value"])
#     keywords_l.append(tuple(tuple_list))

#get hits
hits = response["response"]["meta"]["hits"]





The status code is 200 for page 0 of: gamestop
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=gamestop&begin_date=20201101&end_date=20210220&page=0

The status code is 200 for page 1 of: gamestop
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=gamestop&begin_date=20201101&end_date=20210220&page=1


In [7]:
#append to bottom of series practice
#create dataframe
search_df = pd.DataFrame(columns = [
    "abstract",
    "web_url",
    "main_headline",
    "keywords",
    "pub_date",
    "document_type",
    "type_of_material",
    "nyt_id",
    "word_count",
    "search_hits"
    ]) 

search_df["abstract"] = abstract_l
search_df["web_url"] = web_url_l
search_df["main_headline"] = main_headline_l
search_df["keywords"] = keywords_l
search_df["pub_date"] = pub_date_l
search_df["document_type"] = document_type_l
search_df["type_of_material"] = type_of_material_l
search_df["nyt_id"] = nyt_id_l
search_df["word_count"] = word_count_l
search_df["search_hits"] = hits

#show dataframe
search_df.head(3)


#change nyt_id to be the index?

,abstract,web_url,main_headline,keywords,pub_date,document_type,type_of_material,nyt_id,word_count,search_hits
0,Fueled by amateur traders and online enthusias...,https://www.nytimes.com/2021/02/07/business/ga...,The Hopes That Rose and Fell With GameStop,"(GameStop Corporation, Stocks and Bonds, Hedge...",2021-02-07T10:00:06+0000,article,News,nyt://article/4c1bd5ce-5e1f-5338-8c48-de2901c6...,1537,134
1,"On Thursday, the House Financial Services Comm...",https://www.nytimes.com/video/us/politics/1000...,Watch Live: House Lawmakers Hold Hearing on Ga...,"(Reddit Inc, Banking and Financial Institution...",2021-02-18T16:29:55+0000,multimedia,Video,nyt://video/ab720481-8f54-5765-ade4-11d99793fc00,0,134
2,"Keith Gill, known on YouTube as Roaring Kitty ...",https://www.nytimes.com/video/business/1000000...,‘Roaring Kitty’ Says He Will Continue to Inves...,"(GameStop Corporation, Reddit Inc, Gill, Keith...",2021-02-18T21:25:54+0000,multimedia,Video,nyt://video/25e545da-ffc9-560f-8f32-bf3170574654,0,134


In [16]:
# save results to csv
search_df.to_csv("outputs/raw/saved_gs_temp_utf.csv", encoding="UTF-8")


NameError: name 'search_df' is not defined

# Begin Post-Pull Transform

In [39]:
#load csv results
gs_article_df = pd.read_csv("outputs/raw/saved_gs_temp_utf.csv", encoding = "UTF-8", index_col = 0)
gs_article_df.head(3)

,abstract,web_url,main_headline,keywords,pub_date,document_type,type_of_material,nyt_id,word_count,search_hits
0,Fueled by amateur traders and online enthusias...,https://www.nytimes.com/2021/02/07/business/ga...,The Hopes That Rose and Fell With GameStop,"('GameStop Corporation', 'Stocks and Bonds', '...",2021-02-07T10:00:06+0000,article,News,nyt://article/4c1bd5ce-5e1f-5338-8c48-de2901c6...,1537,134
1,"On Thursday, the House Financial Services Comm...",https://www.nytimes.com/video/us/politics/1000...,Watch Live: House Lawmakers Hold Hearing on Ga...,"('Reddit Inc', 'Banking and Financial Institut...",2021-02-18T16:29:55+0000,multimedia,Video,nyt://video/ab720481-8f54-5765-ade4-11d99793fc00,0,134
2,"Keith Gill, known on YouTube as Roaring Kitty ...",https://www.nytimes.com/video/business/1000000...,‘Roaring Kitty’ Says He Will Continue to Inves...,"('GameStop Corporation', 'Reddit Inc', 'Gill, ...",2021-02-18T21:25:54+0000,multimedia,Video,nyt://video/25e545da-ffc9-560f-8f32-bf3170574654,0,134


In [40]:
# Reform dates to add "ending on week x"
# name "weekly_container"
raw_time = gs_article_df["pub_date"].tolist()
week_ending_l = []


In [41]:
for d in raw_time:
    date_var = datetime.fromisoformat(d[0:-5]).date()
    week_start = date_var - timedelta(days = date_var.weekday())
    week_end = week_start + timedelta(days=6)
    week_ending_l.append(week_end)
gs_article_df["week_ending_in"] = week_ending_l
gs_article_df.head(6)

,abstract,web_url,main_headline,keywords,pub_date,document_type,type_of_material,nyt_id,word_count,search_hits,week_ending_in
0,Fueled by amateur traders and online enthusias...,https://www.nytimes.com/2021/02/07/business/ga...,The Hopes That Rose and Fell With GameStop,"('GameStop Corporation', 'Stocks and Bonds', '...",2021-02-07T10:00:06+0000,article,News,nyt://article/4c1bd5ce-5e1f-5338-8c48-de2901c6...,1537,134,2021-02-07
1,"On Thursday, the House Financial Services Comm...",https://www.nytimes.com/video/us/politics/1000...,Watch Live: House Lawmakers Hold Hearing on Ga...,"('Reddit Inc', 'Banking and Financial Institut...",2021-02-18T16:29:55+0000,multimedia,Video,nyt://video/ab720481-8f54-5765-ade4-11d99793fc00,0,134,2021-02-21
2,"Keith Gill, known on YouTube as Roaring Kitty ...",https://www.nytimes.com/video/business/1000000...,‘Roaring Kitty’ Says He Will Continue to Inves...,"('GameStop Corporation', 'Reddit Inc', 'Gill, ...",2021-02-18T21:25:54+0000,multimedia,Video,nyt://video/25e545da-ffc9-560f-8f32-bf3170574654,0,134,2021-02-21
3,The questions lawmakers should ask at a hearin...,https://www.nytimes.com/2021/02/18/business/de...,What We Want to Know About GameStop,"('GameStop Corporation', 'Robinhood Financial ...",2021-02-18T12:41:20+0000,article,News,nyt://article/b0fc0046-318f-586b-95ac-84cfd3ce...,1845,134,2021-02-21
4,What began as a story about Reddit versus Wall...,https://www.nytimes.com/2021/02/01/podcasts/th...,The GameStop Rebellion,"('Computer and Video Games', 'Computers and th...",2021-02-01T11:00:07+0000,article,News,nyt://article/03eb4ef5-fc02-5eda-9ce8-6bf36413...,417,134,2021-02-07
5,The trader who pumped up the stock on the inte...,https://www.nytimes.com/2021/02/12/business/ga...,Reddit’s ‘Roaring Kitty’ Will Speak at GameSto...,"('Short Selling', 'Hedge Funds', 'Citadel Inve...",2021-02-13T02:21:54+0000,article,News,nyt://article/75554d0f-dec6-5b58-8f7f-f90b11a8...,391,134,2021-02-14


In [42]:
# save results to csv
gs_article_df.to_csv("outputs/cleaned/saved_gs_cleaned.csv", encoding="UTF-8", index = False)